In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_569054/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_49_9,0.995128,0.657555,0.591752,0.991787,0.001580,0.203290,0.285986,0.001520,0.018319,0.039753,0.990181,0.040062,142.900330,329.290069,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_25_3,0.995110,0.644182,0.560948,0.993961,0.001586,0.211229,0.307565,0.002159,0.018168,0.039827,0.990144,0.040137,142.892836,329.282575,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_25_2,0.995102,0.644068,0.560640,0.994344,0.001589,0.211296,0.307781,0.002021,0.018212,0.039861,0.990127,0.040171,142.889446,329.279185,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_25_4,0.995087,0.644572,0.561186,0.993559,0.001594,0.210997,0.307399,0.002302,0.018159,0.039921,0.990097,0.040231,142.883451,329.273190,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_25_1,0.995087,0.644012,0.560460,0.994597,0.001594,0.211330,0.307907,0.001931,0.018264,0.039921,0.990097,0.040232,142.883427,329.273166,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,model_1_1_4,0.795908,0.514236,0.556413,0.630284,0.066203,0.288371,0.310742,0.109423,0.124656,0.257300,0.588626,0.259302,135.430057,321.819796,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
486,model_1_1_3,0.788135,0.521788,0.547140,0.618000,0.068724,0.283887,0.317238,0.113059,0.121853,0.262153,0.572959,0.264194,135.355302,321.745042,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
487,model_1_1_2,0.782314,0.475466,0.457224,0.646783,0.070613,0.311386,0.380226,0.104540,0.130270,0.265730,0.561226,0.267798,135.301093,321.690832,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
488,model_1_1_1,0.775913,0.462566,0.434578,0.653868,0.072689,0.319044,0.396090,0.102443,0.134132,0.269608,0.548326,0.271707,135.243138,321.632877,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
